# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

> The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts. During these phone campaigns, an attractive long-term deposit application, with good interest rates, was offered. For each contact, a large number of attributes was stored (e.g. see Table 2) and if there was a success (the target variable). For the whole database considered, there were 6499 successes (8% success rate).

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [3]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.'','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Some modifications I can see that need to be done:

- Change of 'y' to bool
- Transform 'month' and 'day_of_wek' to int64 (1-12 and 1-7 respectively)
- Create dummies for 'poutcome', 'contact', 'loan' (could be changed to bool with deleting rows with 'unknown'), 'default', 'housing', (same for both), 'education', and 'marital'.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

Business objective is to increase efficiency of directed campaigns for long-term deposit subscriptions by reducing the number of contacts to do. Applying to this project, this can be done by producing a model that can predict if a contact is likely to produce a positive result - applying to this campaign, subscribing to a deposit - and therefore decrease number of calls ending up in negative results.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features (columns 1 - 7), prepare the features and target column for modeling with appropriate encoding and transformations.

In [5]:
dummies_list = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'month', 'day_of_week']
df_dummies = pd.get_dummies(df[dummies_list])
df_dummies['age'] = df['age']

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [6]:
y = df['y'].map({'no':False, 'yes':True})

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_dummies, y)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [8]:
df['y'].value_counts()['yes'] / df['y'].value_counts().sum()

0.11265417111780131

From the gathered data, slightly more than 11.2% of campaign clients produced positive result

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [9]:
logreg = LogisticRegression(max_iter=1000).fit(X_train, y_train)

### Problem 9: Score the Model

What is the accuracy of your model?

In [10]:
train_preds = logreg.predict(X_train) >= 0.5
train_score = accuracy_score(train_preds, y_train)
test_preds = logreg.predict(X_test) >= 0.5
test_score = accuracy_score(test_preds, y_test)
print(train_score, test_score)

0.8862775565698747 0.8870544818879286


As we can see, the accuracy score is 88%.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [11]:
%timeit logreg = LogisticRegression(max_iter=1000).fit(X_train, y_train)

3.25 s ± 671 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
results = [['logreg', '3250', train_score, test_score]]

In [13]:
%timeit knn = KNeighborsClassifier().fit(X_train, y_train)

33.9 ms ± 4.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
# KNN
knn = KNeighborsClassifier().fit(X_train, y_train)
train_preds = knn.predict(X_train)
train_score = accuracy_score(train_preds, y_train)
test_preds = knn.predict(X_test)
test_score = accuracy_score(test_preds, y_test)
results.append(['knn', '33.9', train_score, test_score])

In [15]:
%timeit dtc = DecisionTreeClassifier().fit(X_train, y_train)

309 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# Decision tree
dtc = DecisionTreeClassifier().fit(X_train, y_train)
train_preds = dtc.predict(X_train)
train_score = accuracy_score(train_preds, y_train)
test_preds = dtc.predict(X_test)
test_score = accuracy_score(test_preds, y_test)
results.append(['dtc', '309', train_score, test_score])

In [17]:
%timeit svc = SVC().fit(X_train, y_train)

30.5 s ± 7.69 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# SVC
svc = SVC().fit(X_train, y_train)
train_preds = svc.predict(X_train)
train_score = accuracy_score(train_preds, y_train)
test_preds = svc.predict(X_test)
test_score = accuracy_score(test_preds, y_test)
results.append(['svc', '30500', train_score, test_score])

In [19]:
results

[['logreg', '3', 0.8862775565698747, 0.8870544818879286],
 ['knn', '38.9', 0.9008125343951313, 0.8810333106730116],
 ['dtc', '665', 0.9732931921919006, 0.8329610566184326],
 ['svc', '33100', 0.8873782007704509, 0.8872487132174419]]

In [20]:
# Train time is measured in ms
comparison = pd.DataFrame(results, columns=['Model', 'Train Time', 'Train Accuracy', 'Test Accuracy'])
comparison = comparison.set_index('Model')
comparison

,Train Time,Train Accuracy,Test Accuracy
Model,,,
logreg,3,0.886278,0.887054
knn,38.9,0.900813,0.881033
dtc,665,0.973293,0.832961
svc,33100,0.887378,0.887249


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

Gender is not a feature in the initial dataset, so it is not tested

In [21]:
# Function for calculating score of each model
def showScore(model):
    model_train_score = model.score(X_train, y_train)
    model_test_score = model.score(X_test, y_test)
    print('Train Score:', model_train_score)
    print('Test Score:', model_test_score)
    print('Best Params', model.best_params_)

In [22]:
# Playing with hyperparameters and gridSearchCV: KNN
params = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15]}
grid_knn = GridSearchCV(knn, param_grid=params).fit(X_train, y_train)

In [23]:
# Find its score
showScore(grid_knn)

Train Score: 0.8904859020426662
Test Score: 0.8862775565698747
Best Params {'n_neighbors': 15}


In [24]:
# Decision Tree Classifier
params = {'max_depth': [1, 3, 5, 7, 9, 11]}
grid_dtc = GridSearchCV(dtc, param_grid=params).fit(X_train, y_train)

In [25]:
showScore(grid_dtc)

Train Score: 0.8873782007704509
Test Score: 0.8872487132174419
Best Params {'max_depth': 1}


In [26]:
# SVC
params = {'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
         'gamma': [0.1, 1.0, 10.0, 100.0],}
grid_svc = GridSearchCV(svc, param_grid=params).fit(X_train, y_train)


I got tired after waiting for this to compute for an hour

In [ ]:
showScore(grid_svc)

Results:

|   **Model** 	| **Train Time**, ms 	| **Train Accuracy** 	| **Test Accuracy** 	|
|------------:	|-------------------:	|-------------------:	|------------------:	|
|    logreg   	|        3250        	|      0.886213      	|      0.887540     	|
|     knn     	|        33.9        	|      0.898741      	|      0.883267     	|
|     dtc     	|        310        	|      0.972710      	|      0.837040     	|
|     svc     	|        30500       	|      0.887022      	|      0.888317     	|
|   grid_knn 	|        33.9         	|      0.889255      	|      0.887540     	|
|   grid_dtc  	|        310           	|      0.887022      	|      0.888316     	|
|   grid_svc  	|        ---         	|         ---         	|         ---          	|

As we can see, all models reach accuracy of 88.7-88.8 %%, with anomalous exception of regular DTC.

### Conclusion

All 6 of the models shown resulted in accuracy approaching 90%, which is much higher than 11% of the initial sample. Any of them would do fine, although I suspect that KNN with hyperparameter of 15 would have the best combination of speed and accuracy.